# Roulette ‘0’ Pocket Calibration & Overlay (Template Matching)

1) **Calibration (setup video)**: track the **‘0’ pocket** with template matching, save points, fit a **circle/ellipse**, save calibration.

2) **Overlay (new image/video)**: load the calibration and overlay the fitted curve onto new frames.

> Assumes a similar camera setup (same framing / minimal movement).

## 0) Setup

Install requirements (if needed):

```bash
pip install opencv-python numpy
```

Put your templates (cropped images of the `0` pocket) in a folder, e.g. `templates_zero/`.

In [1]:

import os
import cv2
import math
import json
import numpy as np
from pathlib import Path
from collections import deque

# -------------------------
# USER PATHS (EDIT THESE)
# -------------------------

# Output files
OUT_DIR = Path("roulette_calibration_out")
OUT_DIR.mkdir(exist_ok=True, parents=True)

TRACK_CSV_PATH    = OUT_DIR / "zero_track_points.csv"
CALIB_JSON_PATH   = OUT_DIR / "zero_path_calibration.json"
OVERLAY_VIDEO_OUT = OUT_DIR / "overlay_output.mp4"

# -------------------------
# TRACKING PARAMS (TUNE)
# -------------------------
MIN_SCORE    = 0.60   # NCC threshold (0..1). Raise if you get false matches.
MAX_JUMP     = 50     # Max pixel jump between consecutive accepted detections.
SMOOTH_K     = 3      # Deque length for light smoothing (median).
FRAME_STEP   = 1      # Process every Nth frame (1=all frames).
SEARCH_ROI   = None   # Optional: (x, y, w, h) to restrict search, else None.
DEBUG_PREVIEW = False # True to show a live debug window (may not work in all notebook setups).


## 1) Helper Functions

- `cv2.matchTemplate` with `TM_CCOEFF_NORMED` (normalized cross-correlation)
- best match across all templates
- jump filter + median smoothing
- ellipse fit via `cv2.fitEllipse` (needs ≥ 5 points), else circle fit


In [2]:

def load_templates(folder_path: str):
    folder = Path(folder_path)
    if not folder.exists():
        raise FileNotFoundError(f"Templates folder not found: {folder.resolve()}")

    exts = (".png", ".jpg", ".jpeg", ".bmp", ".webp")
    template_paths = [p for p in sorted(folder.iterdir()) if p.suffix.lower() in exts]
    if not template_paths:
        raise ValueError(f"No template images found in: {folder.resolve()}")

    templates = []
    for p in template_paths:
        img = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        templates.append((p.name, img))

    if not templates:
        raise ValueError("Templates could not be loaded (all read as None).")
    return templates


def best_template_match(frame_gray: np.ndarray, templates):
    best = None
    for name, tmpl in templates:
        if frame_gray.shape[0] < tmpl.shape[0] or frame_gray.shape[1] < tmpl.shape[1]:
            continue

        res = cv2.matchTemplate(frame_gray, tmpl, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, max_loc = cv2.minMaxLoc(res)

        if best is None or max_val > best["score"]:
            best = {
                "name": name,
                "score": float(max_val),
                "top_left": (int(max_loc[0]), int(max_loc[1])),
                "w": int(tmpl.shape[1]),
                "h": int(tmpl.shape[0]),
            }
    return best


def _dist(p1, p2):
    return math.hypot(p1[0]-p2[0], p1[1]-p2[1])


def track_zero_points(
    video_path: str,
    templates_dir: str,
    min_score: float = 0.60,
    max_jump: float = 80,
    smooth_k: int = 5,
    frame_step: int = 1,
    search_roi=None,
    debug_preview: bool = False
):
    templates = load_templates(templates_dir)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Could not open video: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    points = []
    recent = deque(maxlen=max(1, smooth_k))
    prev = None

    frame_idx = -1
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        frame_idx += 1

        if frame_step > 1 and (frame_idx % frame_step != 0):
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # ROI crop (optional)
        roi_offset = (0, 0)
        gray_search = gray
        if search_roi is not None:
            x, y, w, h = search_roi
            x, y = max(0, x), max(0, y)
            w = min(w, width - x)
            h = min(h, height - y)
            gray_search = gray[y:y+h, x:x+w]
            roi_offset = (x, y)

        best = best_template_match(gray_search, templates)
        if best is None:
            continue

        score = best["score"]
        if score < min_score:
            continue

        tl = best["top_left"]
        cx = tl[0] + best["w"] / 2.0 + roi_offset[0]
        cy = tl[1] + best["h"] / 2.0 + roi_offset[1]
        curr = (float(cx), float(cy))

        # Jump filter
        if prev is not None and _dist(prev, curr) > max_jump:
            continue

        recent.append(curr)
        smoothed = (float(np.median([p[0] for p in recent])),
                    float(np.median([p[1] for p in recent])))

        prev = smoothed

        points.append({
            "frame_idx": int(frame_idx),
            "x": smoothed[0],
            "y": smoothed[1],
            "raw_x": curr[0],
            "raw_y": curr[1],
            "score": float(score),
            "template": best["name"],
        })

        if debug_preview:
            vis = frame.copy()
            if search_roi is not None:
                x, y, w, h = search_roi
                cv2.rectangle(vis, (x, y), (x+w, y+h), (0, 255, 255), 2)

            raw_tl = (int(tl[0] + roi_offset[0]), int(tl[1] + roi_offset[1]))
            raw_br = (raw_tl[0] + best["w"], raw_tl[1] + best["h"])
            cv2.rectangle(vis, raw_tl, raw_br, (0, 255, 0), 2)
            cv2.circle(vis, (int(smoothed[0]), int(smoothed[1])), 4, (0, 0, 255), -1)

            cv2.putText(vis, f"score={score:.3f} tmpl={best['name']}",
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            cv2.imshow("debug tracking", vis)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    if debug_preview:
        cv2.destroyAllWindows()

    meta = {
        "fps": float(fps),
        "width": int(width),
        "height": int(height),
        "frame_count": int(frame_count),
        "frame_step": int(frame_step),
    }
    return points, meta


def fit_circle_least_squares(points_xy: np.ndarray):
    x = points_xy[:, 0]
    y = points_xy[:, 1]
    A = np.c_[2*x, 2*y, np.ones_like(x)]
    b = x**2 + y**2
    sol, *_ = np.linalg.lstsq(A, b, rcond=None)
    cx, cy, c = sol
    r = math.sqrt(max(0.0, c + cx**2 + cy**2))
    return float(cx), float(cy), float(r)


def fit_ellipse_or_circle(points_xy: np.ndarray):
    if len(points_xy) >= 5:
        pts = points_xy.astype(np.float32).reshape(-1, 1, 2)
        (cx, cy), (MA, ma), angle = cv2.fitEllipse(pts)  # axis lengths
        return {
            "type": "ellipse",
            "center": [float(cx), float(cy)],
            "axes": [float(MA/2.0), float(ma/2.0)],  # radii
            "angle_deg": float(angle),
        }
    cx, cy, r = fit_circle_least_squares(points_xy)
    return {"type": "circle", "center": [cx, cy], "radius": r}


def draw_calibration_overlay(frame_bgr: np.ndarray, geom: dict, thickness: int = 2):
    out = frame_bgr.copy()
    if geom["type"] == "ellipse":
        cx, cy = geom["center"]
        ax, ay = geom["axes"]
        angle = geom["angle_deg"]
        cv2.ellipse(out, (int(cx), int(cy)), (int(ax), int(ay)), angle, 0, 360, (0, 0, 255), thickness)
    else:
        cx, cy = geom["center"]
        r = geom["radius"]
        cv2.circle(out, (int(cx), int(cy)), int(r), (0, 0, 255), thickness)
    return out


## 2) Calibration Run (Setup Video)

This cell tracks the `0` pocket, saves a CSV of points, fits ellipse/circle, and saves a JSON calibration.

In [3]:
from utils.file_dialog_utils import pick_video_cv2, pick_image_cv2, pick_folder

_, SETUP_VIDEO_PATH = pick_video_cv2(title="Select Input Video")
TEMPLATES_DIR = pick_folder(title="Select Template Folder")

points, meta = track_zero_points(
    video_path=SETUP_VIDEO_PATH,
    templates_dir=TEMPLATES_DIR,
    min_score=MIN_SCORE,
    max_jump=MAX_JUMP,
    smooth_k=SMOOTH_K,
    frame_step=FRAME_STEP,
    search_roi=SEARCH_ROI,
    debug_preview=DEBUG_PREVIEW
)

print(f"Collected {len(points)} points.")
print("Video meta:", meta)


Collected 316 points.
Video meta: {'fps': 30.0, 'width': 970, 'height': 726, 'frame_count': 333, 'frame_step': 1}


In [4]:

# Save points to CSV
import csv

with open(TRACK_CSV_PATH, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["frame_idx", "x", "y", "raw_x", "raw_y", "score", "template"])
    for p in points:
        w.writerow([p["frame_idx"], p["x"], p["y"], p["raw_x"], p["raw_y"], p["score"], p["template"]])

print("Saved:", TRACK_CSV_PATH.resolve())


Saved: C:\Users\Gabriel\Documents\Dissertation\Code\notebooks\roulette_calibration_out\zero_track_points.csv


In [5]:

# Fit geometry + save calibration JSON
pts_xy = np.array([[p["x"], p["y"]] for p in points], dtype=np.float64)
geom = fit_ellipse_or_circle(pts_xy)

import datetime
calibration = {
    "created_utc": datetime.datetime.utcnow().isoformat() + "Z",
    "source_setup_video": str(SETUP_VIDEO_PATH),
    "templates_dir": str(TEMPLATES_DIR),
    "tracking_params": {
        "min_score": MIN_SCORE,
        "max_jump": MAX_JUMP,
        "smooth_k": SMOOTH_K,
        "frame_step": FRAME_STEP,
        "search_roi": SEARCH_ROI,
    },
    "video_meta": meta,
    "geometry": geom,
    "point_count": int(len(points)),
}

with open(CALIB_JSON_PATH, "w") as f:
    json.dump(calibration, f, indent=2)

print("Saved calibration JSON:", CALIB_JSON_PATH.resolve())
print("Geometry:", geom)


Saved calibration JSON: C:\Users\Gabriel\Documents\Dissertation\Code\notebooks\roulette_calibration_out\zero_path_calibration.json
Geometry: {'type': 'ellipse', 'center': [488.2545166015625, 394.57574462890625], 'axes': [256.97064208984375, 275.6770935058594], 'angle_deg': 91.70647430419922}


C:\Users\Gabriel\AppData\Local\Temp\ipykernel_5712\913877172.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_utc": datetime.datetime.utcnow().isoformat() + "Z",


## 3) Quick sanity check

This draws the fitted path on the first frame of the setup video and saves an image preview.

In [6]:

cap = cv2.VideoCapture(SETUP_VIDEO_PATH)
ok, frame0 = cap.read()
cap.release()

if not ok:
    raise RuntimeError("Could not read first frame for preview.")

preview = draw_calibration_overlay(frame0, geom, thickness=3)
preview_path = OUT_DIR / "calibration_preview.png"
cv2.imwrite(str(preview_path), preview)
print("Wrote preview image:", preview_path.resolve())


Wrote preview image: C:\Users\Gabriel\Documents\Dissertation\Code\notebooks\roulette_calibration_out\calibration_preview.png


## 4) Overlay onto a new image or video

Set `NEW_IMAGE_PATH` and/or `NEW_VIDEO_PATH` and run the next cells.

In [7]:

# Load calibration (useful if you restart the notebook)
with open(CALIB_JSON_PATH, "r") as f:
    calibration = json.load(f)
geom = calibration["geometry"]

_ , NEW_IMAGE_PATH = pick_image_cv2(title="Select New Overlay Image (or Cancel to skip)")
_ , NEW_VIDEO_PATH = pick_video_cv2(title="Select New Overlay Video (or Cancel to skip)")

print("Loaded geometry:", geom)


Loaded geometry: {'type': 'ellipse', 'center': [488.2545166015625, 394.57574462890625], 'axes': [256.97064208984375, 275.6770935058594], 'angle_deg': 91.70647430419922}


In [8]:

# Image overlay
if NEW_IMAGE_PATH and os.path.exists(NEW_IMAGE_PATH):
    img = cv2.imread(NEW_IMAGE_PATH)
    if img is None:
        raise RuntimeError("Could not read NEW_IMAGE_PATH")
    out = draw_calibration_overlay(img, geom, thickness=3)
    out_path = OUT_DIR / "overlay_image.png"
    cv2.imwrite(str(out_path), out)
    print("Saved overlay image:", out_path.resolve())
else:
    print("Skipping image overlay (set NEW_IMAGE_PATH to an existing file).")


Saved overlay image: C:\Users\Gabriel\Documents\Dissertation\Code\notebooks\roulette_calibration_out\overlay_image.png


In [9]:

# Video overlay
if NEW_VIDEO_PATH and os.path.exists(NEW_VIDEO_PATH):
    cap = cv2.VideoCapture(NEW_VIDEO_PATH)
    if not cap.isOpened():
        raise RuntimeError("Could not open NEW_VIDEO_PATH")

    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(str(OVERLAY_VIDEO_OUT), fourcc, fps, (width, height))

    while True:
        ok, frame = cap.read()
        if not ok:
            break
        overlay = draw_calibration_overlay(frame, geom, thickness=2)
        writer.write(overlay)

    cap.release()
    writer.release()
    print("Saved overlay video:", OVERLAY_VIDEO_OUT.resolve())
else:
    print("Skipping video overlay (set NEW_VIDEO_PATH to an existing file).")


Saved overlay video: C:\Users\Gabriel\Documents\Dissertation\Code\notebooks\roulette_calibration_out\overlay_output.mp4


## Optional: Make it more robust

If your camera framing shifts between videos, you can:
- detect stable landmarks (rim / center hub)
- estimate a transform (similarity/homography)
- warp the calibration geometry before drawing
